### Imports


In [1]:
import os,csv
import numpy as np
import tensorflow as tf
import keras
from mtcnn import MTCNN
from numpy import asarray
import matplotlib.pyplot as plt

from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from PIL import Image
import hashlib
from flask import Flask, request, jsonify
import io
import threading
from skimage.filters import threshold_otsu


### Model definition


In [2]:
app = Flask(__name__)

def extract_face(pixels, required_size=(224, 224)):

    detector = MTCNN()
	
    results = detector.detect_faces(pixels)
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
	
    face = pixels[y1:y2, x1:x2]
	
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    face_array = face_array.astype('float32')
    
    img_array = np.expand_dims(face_array, axis=0)
    img_array = preprocess_input(img_array, version=2)
    
    return img_array

def hash_features(features, output_dim=64):
    hasher = hashlib.sha256()
    hasher.update(features.tobytes())
    hash_digest = hasher.digest()
    # To get 64 bytes, concatenate two hash digests
    extended_hash = hash_digest + hashlib.sha256(hash_digest).digest()
    
    # Convert the extended hash to binary representation
    binary_hash = ''.join(format(byte, '08b') for byte in extended_hash)
    
    # Ensure the output_dim does not exceed the length of the binary hash
    binary_hash = binary_hash[:output_dim]
    
    # Convert the binary string to a numpy array of 0s and 1s
    hash_array = np.array(list(binary_hash), dtype=int)
    
    return hash_array


def extract_features(img):
    img_array = extract_face(img)

    # Charger le modèle SENet50 pré-entraîné
    model = VGGFace(model='senet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

    # Extraire les features de l'image
    features = model.predict(img_array)
    # hashed_features = hash_features(features, output_dim=64)
    return features


@app.route('/extract_features', methods=['POST'])
def extract_features_route():
    if 'image' not in request.files:
        return jsonify({'error': 'No image part in the request'}), 400

    file = request.files['image']

    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    try:
        image = Image.open(file.stream)
        img=np.array(image)
        features = extract_features(img)
        return {"features": features.tolist()}
    except Exception as e:
        return jsonify({'error': str(e)}), 500

### Lunching the model


In [4]:
def run_app():
    app.run(debug=False, use_reloader=False)

# Run Flask app in a separate thread
thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


## Test 

In [ ]:
import requests

url = 'http://127.0.0.1:5000/extract_features'
# replace this with your image path you wanna to extract features for
files = {'image': open(r'C:\Users\LENOVO\Pictures\WIN_20230220_17_14_34_Pro.jpg', 'rb')}

response = requests.post(url, files=files)
print(response.json())


1/1 [==============================] - 1s 1s/step


127.0.0.1 - - [28/Nov/2024 12:52:46] "POST /extract_features HTTP/1.1" 200 -


{'features': [[0.0356370247900486, 0.676445722579956, 0.012586538679897785, 0.0035092842299491167, 0.288446843624115, 3.5982301235198975, 0.07672320306301117, 0.055334124714136124, 1.1984282732009888, 11.993590354919434, 1.2766616344451904, 7.388052940368652, 0.9482169151306152, 0.0030507217161357403, 0.26934608817100525, 0.0021314017940312624, 9.455588340759277, 0.00460653705522418, 0.00633635651320219, 3.4574615955352783, 5.0260114669799805, 1.686262845993042, 0.009124555625021458, 0.25714099407196045, 0.0015411439817398787, 0.07822541147470474, 2.347466468811035, 0.0017973312642425299, 0.06611600518226624, 0.01972905918955803, 0.4204868674278259, 0.04799087718129158, 0.038369715213775635, 0.46290889382362366, 0.02748984657227993, 0.0500885434448719, 0.010065220296382904, 8.469816207885742, 0.19958968460559845, 2.320176839828491, 0.011101175099611282, 0.1533684879541397, 0.013899027369916439, 8.292081832885742, 0.0041404482908546925, 0.00021101615857332945, 0.0016304010059684515, 4.7